In [ ]:
from pyspark.sql import SparkSession
from random import randint
from pyspark.sql.types import IntegerType, StructField, StructType
import pyspark.sql.functions as F
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
#from sklearn.model_selection import train_test_split

In [ ]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

In [ ]:
spark = SparkSession.builder.appName('SparkTitanic.com').getOrCreate()

Reading the data

In [ ]:
train_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/train.csv')

Cleaning the data

In [ ]:
train_df = train_df.select("Pclass", "Sex", "SibSp", "Parch", "Survived")
train_df = train_df.where(train_df["Pclass"].isNotNull())
train_df = train_df.where(train_df["Sex"].isNotNull())
train_df = train_df.where(train_df["SibSp"].isNotNull())
train_df = train_df.where(train_df["Parch"].isNotNull())
train_df = train_df.where(train_df["Survived"].isNotNull())
train_df = train_df.withColumn("Sex", F.when(F.col("Sex")==F.lit("male"), 0.).otherwise(1.) )

In [ ]:
x_train = list(map(lambda x: list(x), train_df.select("Pclass", "Sex", "SibSp", "Parch").collect()))

In [ ]:
x_train = np.array([[(x[0]-1)/2, x[1], x[2], x[3]/5] for x in x_train], dtype="float32")

In [ ]:
y_train = list(map(lambda x: list(x)[0], train_df.select("Survived").collect()))

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.30, random_state=40)

In [ ]:
model = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2)

In [ ]:
model.fit(x_train, y_train)

In [ ]:
#y_pred_test = model.predict(x_test)

In [ ]:
#accuracy_score(y_test, y_pred_test)

Prediction

In [ ]:
def prediction(row): #"Pclass", "Sex", "SibSp", "Parch"
    Pclass = row.Pclass
    if Pclass == None:
        Pclass = randint(0,2)
    Pclass = (Pclass-1)/2

    if row.Sex == 'female':
        Sex = 1.
    elif row.Sex == 'male':
        Sex = 0.
    else:
        Sex = randint(0,1)*1.

    SibSp = row.SibSp
    if SibSp == None:
        SibSp = randint(0,1)
    
    Parch = row.Parch
    if Parch == None:
        Parch = randint(0,2)
    Parch = (Parch-1)/2

    p = model.predict([ [Pclass, Sex, SibSp, Parch] ])[0]
    return int(p)

In [ ]:
test_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/test.csv')

In [ ]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize([]), StructType([
    StructField('PassengerId', IntegerType(), True),
    StructField('Survived', IntegerType(), True)
]))

In [ ]:
for row in test_df.collect():
    output_df = output_df.union( spark.sparkContext.parallelize([(row.PassengerId, prediction(row) )]).toDF(['PassengerId', 'Survived']) )

Output

In [ ]:
output_df.toPandas().to_csv('./componentA.csv', index=False)